In [2]:
from itertools import chain
import pandas as pd
import nltk
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import glob
import sklearn
import pycrfsuite
import os
import sys
sys.path.append("..")
from Scripts.util import Preprocessing,SentenceSplitter
from sinling import SinhalaTokenizer
import kaggle
print(sklearn.__version__)

1.5.1


In [3]:
from kaggle.api.kaggle_api_extended import KaggleApi


kaggle.api.authenticate()

kaggle.api.dataset_download_files("kavishaabeynayaka/sinhala-fg-ner", path='../AnanyaSinhalaNERDataset/combined_data', unzip=True)





Dataset URL: https://www.kaggle.com/datasets/kavishaabeynayaka/sinhala-fg-ner


In [4]:

# Step 1: Load multiple TSV files
file_paths = glob.glob('/home/kavisha/Research/AnanyaSinhalaNERDataset/combined_data/*.tsv')  #


print("Files found:", file_paths)  


if not file_paths:
    print("No TSV files found. Please check the path.")
else:

    dataframes = []
    for file in file_paths:
        df = pd.read_csv(file, sep='\t',names=["Token","Postag","Nertag"])
        preprocessing = Preprocessing(df)
        cleaned_df = preprocessing.apply_preprocessing()
  
        if not df.empty:  
            dataframes.append(cleaned_df)
        else:
            print(f"Skipping empty file: {file}")
    print(dataframes)
    if dataframes:
        data = pd.concat(dataframes, axis=0)
        print(data.shape)
        train_data, temp_data = train_test_split(data, test_size=0.4,shuffle=False)
     
        val_data, test_data = train_test_split(temp_data, test_size=0.5,shuffle=False)

       
        print("Training set size:", len(train_data))
        print("Validation set size:", len(val_data))
        print("Testing set size:", len(test_data))

        train_data.to_csv('/home/kavisha/Research/combined_data/train.tsv', sep='\t', index=False)
        val_data.to_csv('/home/kavisha/Research/combined_data/validation.tsv', sep='\t', index=False)
        test_data.to_csv('/home/kavisha/Research/combined_data/est.tsv', sep='\t', index=False)
    else:
        print("No valid dataframes to concatenate.")


Files found: ['/home/kavisha/Research/AnanyaSinhalaNERDataset/combined_data/partition_13_combined.tsv', '/home/kavisha/Research/AnanyaSinhalaNERDataset/combined_data/partition_10_combined.tsv', '/home/kavisha/Research/AnanyaSinhalaNERDataset/combined_data/partition_4_combined.tsv', '/home/kavisha/Research/AnanyaSinhalaNERDataset/combined_data/partition_14_combined.tsv', '/home/kavisha/Research/AnanyaSinhalaNERDataset/combined_data/partition_1_combined.tsv', '/home/kavisha/Research/AnanyaSinhalaNERDataset/combined_data/partition_11_combined.tsv', '/home/kavisha/Research/AnanyaSinhalaNERDataset/combined_data/partition_7_combined.tsv', '/home/kavisha/Research/AnanyaSinhalaNERDataset/combined_data/partition_12_combined.tsv', '/home/kavisha/Research/AnanyaSinhalaNERDataset/combined_data/partition_5_combined.tsv', '/home/kavisha/Research/AnanyaSinhalaNERDataset/combined_data/partition_8_combined.tsv', '/home/kavisha/Research/AnanyaSinhalaNERDataset/combined_data/partition_15_combined.tsv', '

Features


In [5]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag,
        'postag[:2]=' + postag[:2],
    ]
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:postag=' + postag1,
            '-1:postag[:2]=' + postag1[:2],
        ])
    else:
        features.append('BOS')
        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:postag=' + postag1,
            '+1:postag[:2]=' + postag1[:2],
        ])
    else:
        features.append('EOS')
                
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]  

In [16]:
train_data = pd.read_csv("/home/kavisha/Research/combined_data/train.tsv",sep="\t")
train_data.shape
train_data.head
train_splitter =  SentenceSplitter(train_data)



In [18]:
def prepare_sentences(df):
    # Remove rows where 'Token' or 'Postag' columns contain empty strings
    # Drop rows with any NaN values
    df = df.dropna()

    sentences = []
    for _, sentence_df in df.groupby("SentenceID"):
        sentence = list(sentence_df[['Token', 'Postag', 'Nertag']].itertuples(index=False, name=None))
        sentences.append(sentence)
    return sentences

train_sents =  prepare_sentences(train_splitter.split_sentences())

X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

# X_test = [sent2features(s) for s in test_sents]
# y_test = [sent2labels(s) for s in test_sents]


In [22]:
%%time
trainer = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

CPU times: user 717 ms, sys: 3.66 ms, total: 721 ms
Wall time: 720 ms


In [23]:
trainer.set_params({
    'c1': 1.0,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 50,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})


In [21]:
trainer.params()

['feature.minfreq',
 'feature.possible_states',
 'feature.possible_transitions',
 'c1',
 'c2',
 'max_iterations',
 'num_memories',
 'epsilon',
 'period',
 'delta',
 'linesearch',
 'max_linesearch']

In [25]:
trainer.train('SinBaseCRF.crfsuite')